In [2]:
import gurobipy as gp
from gurobipy import GRB
import itertools as it

In [67]:
blocks = [(i, j, k) for i in range(1, 4) for j in range(1, 4) for k in range(1, 4)]

lines_diag = []

for i in range(1, 4):
    for j in range(1, 4):
        for k in range (1, 4):
            if i == 1:
                lines_diag.append(((1,j,k), (2,j,k), (3,j,k)))
            if j == 1:
                lines_diag.append(((i,1,k), (i,2,k), (i,3,k)))
            if k == 1:
                lines_diag.append(((i,j,1), (i,j,2), (i,j,3)))
            if i == 1 and j == 1:
                lines_diag.append(((1,1,k), (2,2,k), (3,3,k)))
            if i == 1 and j == 3:
                lines_diag.append(((1,3,k), (2,2,k), (3,1,k)))
            if i == 1 and k == 1:
                lines_diag.append(((1,j,1), (2,j,2), (3,j,3)))
            if i == 1 and k == 3:
                lines_diag.append(((1,j,3), (2,j,2), (3,j,1)))
            if j == 1 and k == 1:
                lines_diag.append(((i,1,1), (i,2,2), (i,3,3)))
            if j == 1 and k == 3:
                lines_diag.append(((i,1,3), (i,2,2), (i,3,1)))
lines_diag.append(((1,1,1), (2,2,2), (3,3,3)))
lines_diag.append(((3,1,1), (2,2,2), (1,3,3)))
lines_diag.append(((1,3,1), (2,2,2), (2,1,2)))
lines_diag.append(((1,1,3), (2,2,2), (3,3,1)))

In [119]:
model = gp.Model('3D Noughts Crosses')

# add vars
cross = model.addVars(blocks,
                      vtype=GRB.BINARY,
                      name='cross')
line = model.addVars(lines_diag,
                     vtype=GRB.BINARY,
                     name='line')

# objective function
model.setObjective(gp.quicksum(line))

# add constrs
model.addConstr(gp.quicksum(cross) == 14)
model.addConstrs(cross[l[0]] + cross[l[1]] + cross[l[2]] - line[l] <= 2 for l in line)
model.addConstrs(cross[l[0]] + cross[l[1]] + cross[l[2]] + line[l] >= 1 for l in line)  # according to the book, and z=3
model.addConstrs(cross[l[0]] + cross[l[1]] + cross[l[2]] - line[l] >= 1 for l in line)  # to find the correct answer, z=4

model.update()

In [120]:
model.write('3D Noughts Crosses.lp')
model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 99 rows, 76 columns and 419 nonzeros
Model fingerprint: 0x6a643572
Variable types: 0 continuous, 76 integer (76 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Found heuristic solution: objective 5.0000000
Presolve time: 0.00s
Presolved: 99 rows, 76 columns, 419 nonzeros
Variable types: 0 continuous, 76 integer (76 binary)

Root relaxation: objective 0.000000e+00, 55 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    8    5.00000    0.00000   100%     -    0s
H    0     0                       4.0000000    0.00000   100%     -    0s
     0     0    0.00000    0  

In [107]:
for c in cross:
    if cross[c].x == 1.0:
        print(c)

(1, 1, 2)
(1, 2, 2)
(1, 2, 3)
(1, 3, 1)
(1, 3, 3)
(2, 1, 1)
(2, 1, 3)
(2, 2, 1)
(2, 2, 2)
(2, 3, 2)
(3, 1, 2)
(3, 1, 3)
(3, 2, 1)
(3, 3, 3)


In [118]:
for c in line:
    #if line[c].x == 1.0:
    print(c, line[c].x)
    print(sum(cross[block].x for block in c))

((1, 1, 1), (2, 1, 1), (3, 1, 1)) 0.0
1.0
((1, 1, 1), (1, 2, 1), (1, 3, 1)) 0.0
1.0
((1, 1, 1), (1, 1, 2), (1, 1, 3)) 0.0
1.0
((1, 1, 1), (2, 2, 1), (3, 3, 1)) 0.0
1.0
((1, 1, 1), (2, 1, 2), (3, 1, 3)) 0.0
1.0
((1, 1, 1), (1, 2, 2), (1, 3, 3)) 0.0
2.0
((1, 1, 2), (2, 1, 2), (3, 1, 2)) 0.0
2.0
((1, 1, 2), (1, 2, 2), (1, 3, 2)) 0.0
2.0
((1, 1, 2), (2, 2, 2), (3, 3, 2)) 0.0
2.0
((1, 1, 3), (2, 1, 3), (3, 1, 3)) 0.0
2.0
((1, 1, 3), (1, 2, 3), (1, 3, 3)) 0.0
2.0
((1, 1, 3), (2, 2, 3), (3, 3, 3)) 0.0
1.0
((1, 1, 3), (2, 1, 2), (3, 1, 1)) 1.0
0.0
((1, 1, 3), (1, 2, 2), (1, 3, 1)) 0.0
2.0
((1, 2, 1), (2, 2, 1), (3, 2, 1)) 0.0
2.0
((1, 2, 1), (1, 2, 2), (1, 2, 3)) 0.0
2.0
((1, 2, 1), (2, 2, 2), (3, 2, 3)) 0.0
1.0
((1, 2, 2), (2, 2, 2), (3, 2, 2)) 0.0
2.0
((1, 2, 3), (2, 2, 3), (3, 2, 3)) 0.0
1.0
((1, 2, 3), (2, 2, 2), (3, 2, 1)) 1.0
3.0
((1, 3, 1), (2, 3, 1), (3, 3, 1)) 0.0
1.0
((1, 3, 1), (1, 3, 2), (1, 3, 3)) 0.0
2.0
((1, 3, 1), (2, 2, 1), (3, 1, 1)) 0.0
2.0
((1, 3, 1), (2, 3, 2), (3, 3, 3)) 